In [1]:
from exif import Image
import pandas as pd
import PIL
import os
import zmq
import zlib
import pyulog
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [2]:
port = '5555'
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://0.0.0.0:%s" % port)

def send_zipped_pickle(socket, obj, flags=0, protocol=-1):
    p = pickle.dumps(obj, protocol)
    z = zlib.compress(p)
    return socket.send(z, flags=flags)

In [3]:
buff = {
    "timestamp_tiplot": [],
    "timestamp": [],
    "lat": [],
    "lon": [],
    "alt": [],
    "zero": [],
    "one": [],
}

def image_coordinates(image_path):
    global t
    with open(image_path, 'rb') as src:
        img = Image(src)
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (decimal_coords(img.gps_latitude,
                      img.gps_latitude_ref),
                      decimal_coords(img.gps_longitude,
                      img.gps_longitude_ref))
        except AttributeError:
            print('No Coordinates')
    else:
        print('The Image has no EXIF information')
    timestamp = img.datetime_original
    lat = coords[0]
    lon = coords[1]
    alt = img.gps_altitude
    dt = datetime.strptime(timestamp, '%Y:%m:%d %H:%M:%S')
    
    buff['timestamp_tiplot'].append(dt.timestamp()/1e6)
    buff['timestamp'].append(timestamp)
    buff['lat'].append(lat)
    buff['lon'].append(lon)
    buff['alt'].append(alt)
    
    buff['zero'].append(0) # for qx,qy,qz
    buff['one'].append(1) # for qw
    
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == "S" or ref == "W":
     decimal_degrees = -decimal_degrees
 return decimal_degrees

for f in os.listdir(os.getcwd()+"/photos/"):
    filename = os.getcwd()+"/photos/"+f
    image_coordinates(filename)
    

In [4]:
data=pd.DataFrame.from_dict(buff)
data = data.sort_values(by=['timestamp_tiplot'])
datadict = {"info": data}


In [5]:
uav = {
    "name": "uav",
    "alpha": 1,
    "useRPY": False,
    "useXYZ": False,
    "pathColor": "#00ff1e",
    "wireframe": False,
    "color": "#ffffff",
    "tracked": False,
    "scale": 1,
    "position": {
      "table": "info",
      "longitude": "lon",
      "lattitude": "lat",
      "altitude": "alt"
    },
    "attitude": {
      "table": "info",
      "q0": "one",
      "q1": "zero",
      "q2": "zero",
      "q3": "zero"
    }
  }


In [6]:
send_zipped_pickle(socket, [datadict, [uav]])